In [1]:
import os, sys
import pickle
import numpy as np
from tqdm import tqdm

## Waymo

In [2]:
waymo_anno_dir = "/project/mira/personal/timmy8986/3dal_pytorch/data/Waymo/train/annos"
waymo_lidar_dir = "/project/mira/personal/timmy8986/3dal_pytorch/data/Waymo/train/lidar"

anno_files = os.listdir(waymo_anno_dir)
lidar_files = os.listdir(waymo_lidar_dir)

with open(os.path.join(waymo_anno_dir, anno_files[0]), 'rb') as pk:
    anno = pickle.load(pk)
    print(anno.keys())
    print('scene_name    : ', anno['scene_name'])
    print('frame_name    : ', anno['frame_name'])
    print('frame_id      : ', anno['frame_id'])
    print('veh_to_global : \n', anno['veh_to_global'])
    print('objects[0]    : \n', anno['objects'][0])

print('\n\n')
with open(os.path.join(waymo_lidar_dir, lidar_files[0]), 'rb') as pk:
    lidar = pickle.load(pk)
    print(lidar.keys())
    print('scene_name : ', lidar['scene_name'])
    print('frame_name : ', lidar['frame_name'])
    print('frame_id   : ', lidar['frame_id'])
    print('lidars     : \n', lidar['lidars'])

dict_keys(['scene_name', 'frame_name', 'frame_id', 'veh_to_global', 'objects'])
scene_name    :  1005081002024129653_5313_150_5333_150
frame_name    :  1005081002024129653_5313_150_5333_150_location_phx_Day_1510593600140139
frame_id      :  0
veh_to_global : 
 [ 9.99868367e-01 -1.62249017e-02  2.35025318e-05  9.27332247e+03
  1.62019584e-02  9.98375338e-01 -5.46276613e-02  9.46533570e+03
  8.62864088e-04  5.46208513e-02  9.98506794e-01 -1.95371000e+02
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]
objects[0]    : 
 {'id': 0, 'name': '8EFRSwEXBf9P-SzIDRzx0A', 'label': 1, 'box': array([-2.0020041e+01,  7.2557716e+00,  5.0248885e-01,  4.7602353e+00,
        1.9828961e+00,  1.6799999e+00,  2.0331686e+01, -9.9822059e-02,
       -9.0812016e-03], dtype=float32), 'num_points': 606, 'detection_difficulty_level': 0, 'combined_difficulty_level': 1, 'global_speed': array([20.33063   ,  0.23041475], dtype=float32), 'global_accel': array([0.22777614, 0.03386023], dtype=float32)}



In [3]:
waymo_labels = {1: 'Vehicle', 2: 'Pedestrian', 4: 'Cyclist'}
waymo_boxes = {1: np.empty((0,3), np.float32), 2: np.empty((0,3), np.float32), 4: np.empty((0,3), np.float32)}
waymo_lwh_mean = {1: np.zeros((1,3), np.float32), 2: np.zeros((1,3), np.float32), 4: np.zeros((1,3), np.float32)}

sample_num = 10000
for anno_file in tqdm(anno_files[:sample_num]):
    with open(os.path.join(waymo_anno_dir, anno_file), 'rb') as pk:
        anno = pickle.load(pk)
        for anno_obj in anno['objects']:
            label = anno_obj['label']
            box = anno_obj['box']
            if label in waymo_labels:
                waymo_boxes[label] = np.vstack((waymo_boxes[label], box[3:6]))

for label in waymo_labels:
    if np.any(waymo_boxes[label]):
        waymo_lwh_mean[label] = np.mean(waymo_boxes[label], axis=0)

print(waymo_lwh_mean)

# sample_num = 10000
# {1: array([4.757425 , 2.0977216, 1.7924396], dtype=float32), 
#  2: array([0.915268 , 0.8720095, 1.714554 ], dtype=float32), 
#  4: array([1.7481283 , 0.82206863, 1.7092806 ], dtype=float32)}

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:13<00:00, 74.71it/s]


{1: array([4.757425 , 2.0977216, 1.7924396], dtype=float32), 2: array([0.915268 , 0.8720095, 1.714554 ], dtype=float32), 4: array([1.7481283 , 0.82206863, 1.7092806 ], dtype=float32)}


## nuScenes

In [4]:
from nuscenes.nuscenes import NuScenes
nusc = NuScenes(version='v1.0-mini', dataroot='/tmp2/tkyen/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.6 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [5]:
nusc.list_categories()

Category stats for split v1.0-mini:
human.pedestrian.adult      n= 4765, width= 0.68±0.11, len= 0.73±0.17, height= 1.76±0.12, lw_aspect= 1.08±0.23
human.pedestrian.child      n=   46, width= 0.46±0.08, len= 0.45±0.09, height= 1.37±0.06, lw_aspect= 0.97±0.05
human.pedestrian.constructi n=  193, width= 0.69±0.07, len= 0.74±0.12, height= 1.78±0.05, lw_aspect= 1.07±0.16
human.pedestrian.personal_m n=   25, width= 0.83±0.00, len= 1.28±0.00, height= 1.87±0.00, lw_aspect= 1.55±0.00
human.pedestrian.police_off n=   11, width= 0.59±0.00, len= 0.47±0.00, height= 1.81±0.00, lw_aspect= 0.80±0.00
movable_object.barrier      n= 2323, width= 2.32±0.49, len= 0.61±0.11, height= 1.06±0.10, lw_aspect= 0.28±0.09
movable_object.debris       n=   13, width= 0.43±0.00, len= 1.43±0.00, height= 0.46±0.00, lw_aspect= 3.35±0.00
movable_object.pushable_pul n=   82, width= 0.51±0.06, len= 0.79±0.10, height= 1.04±0.20, lw_aspect= 1.55±0.18
movable_object.trafficcone  n= 1378, width= 0.47±0.14, len= 0.45±0.07, heigh

In [6]:
print(nusc.sample_annotation[0])

{'token': '70aecbe9b64f4722ab3c230391a3beb8', 'sample_token': 'cd21dbfc3bd749c7b10a5c42562e0c42', 'instance_token': '6dd2cbf4c24b4caeb625035869bca7b5', 'visibility_token': '4', 'attribute_tokens': ['4d8821270b4a47e3a8a300cbec48188e'], 'translation': [373.214, 1130.48, 1.25], 'size': [0.621, 0.669, 1.642], 'rotation': [0.9831098797903927, 0.0, 0.0, -0.18301629506281616], 'prev': 'a1721876c0944cdd92ebc3c75d55d693', 'next': '1e8e35d365a441a18dd5503a0ee1c208', 'num_lidar_pts': 5, 'num_radar_pts': 0, 'category_name': 'human.pedestrian.adult'}


In [7]:
nuscenes_labels = waymo_labels
nuscenes_boxes = {1: np.empty((0,3), np.float32), 2: np.empty((0,3), np.float32), 4: np.empty((0,3), np.float32)}
nuscenes_lwh_mean = {1: np.zeros((1,3), np.float32), 2: np.zeros((1,3), np.float32), 4: np.zeros((1,3), np.float32)}
print(nuscenes_labels)

for sample_annotation in nusc.sample_annotation:
    category_name = sample_annotation['category_name']
    size = np.array(sample_annotation['size'])

    if category_name in ['vehicle.car']:
        nuscenes_boxes[1] = np.vstack((nuscenes_boxes[1], size))
    elif 'human.pedestrian' in category_name:
        nuscenes_boxes[2] = np.vstack((nuscenes_boxes[2], size))
    if category_name in ['vehicle.bicycle', 'vehicle.motorcycle']:
        nuscenes_boxes[4] = np.vstack((nuscenes_boxes[4], size))

for label in nuscenes_labels:
    if np.any(nuscenes_boxes[label]):
        lwh_mean = np.mean(nuscenes_boxes[label], axis=0) # w, l, h
        lwh_mean[[0, 1]] = lwh_mean[[1, 0]]
        nuscenes_lwh_mean[label] = lwh_mean               # l, w, h

print(nuscenes_lwh_mean)

{1: 'Vehicle', 2: 'Pedestrian', 4: 'Cyclist'}
{1: array([4.62030673, 1.92481494, 1.69039835]), 2: array([0.73394802, 0.68270278, 1.75781567]), 4: array([1.90842437, 0.66322129, 1.44645658])}


## Waymo to nuScenes

In [8]:
w2n_anno_dir = "/tmp2/tkyen/3DAL/da/waymo_to_nuscene/train/annos"
w2n_lidar_dir = "/tmp2/tkyen/3DAL/da/waymo_to_nuscene/train/lidar"
w2n_deltas = {1: np.zeros((1,3), np.float32), 2: np.zeros((1,3), np.float32), 4: np.zeros((1,3), np.float32)}

print("waymo_lwh_mean   : \n", waymo_lwh_mean, "\n")
print("nuscenes_lwh_mean: \n", nuscenes_lwh_mean)
print()

for label in waymo_labels:
    w2n_deltas[label] = nuscenes_lwh_mean[label] - waymo_lwh_mean[label]

print(w2n_deltas)

waymo_lwh_mean   : 
 {1: array([4.757425 , 2.0977216, 1.7924396], dtype=float32), 2: array([0.915268 , 0.8720095, 1.714554 ], dtype=float32), 4: array([1.7481283 , 0.82206863, 1.7092806 ], dtype=float32)} 

nuscenes_lwh_mean: 
 {1: array([4.62030673, 1.92481494, 1.69039835]), 2: array([0.73394802, 0.68270278, 1.75781567]), 4: array([1.90842437, 0.66322129, 1.44645658])}

{1: array([-0.1371181 , -0.17290664, -0.10204123]), 2: array([-0.18131999, -0.18930674,  0.04326172]), 4: array([ 0.16029607, -0.15884734, -0.26282403])}


In [10]:
np.set_printoptions(threshold=np.inf, suppress=True)

for anno_file in tqdm(anno_files):
    with open(os.path.join(waymo_anno_dir, anno_file), 'rb') as anno_pk:
        anno = pickle.load(anno_pk)
    with open(os.path.join(waymo_lidar_dir, anno_file), 'rb') as lidar_pk:
        lidar = pickle.load(lidar_pk)
        world_xyz = lidar['lidars']['points_xyz']

    for anno_obj in anno['objects']:
        # for anno
        label = anno_obj['label']
        if label not in waymo_labels:
            continue
        source_box = anno_obj['box']
        target_box = np.copy(source_box)
        target_box[3:6] = target_box[3:6] + w2n_deltas[label]
        anno_obj['box'] = target_box

        # for lidar
        X, Y, Z, Ls, Ws, Hs, _, _, heading_angle = source_box
        heading_vector = np.array([np.cos(heading_angle), np.sin(heading_angle)])
        lateral_vector = np.cross(np.array([0,0,1]), np.append(heading_vector, np.zeros(1)))[:2]

        # Xs, Ys, Zs is the bottom right corner of the bounding box (x: forward / y: left / z: upward)
        Xs, Ys = np.array([X, Y]) - heading_vector * Ls / 2 - lateral_vector * Ws / 2
        Zs = Z - Hs / 2

        # world coordinate system to bounding box coordinate system
        # w = R_wb * b + t_wb
        R_wb = np.array([[np.cos(heading_angle), -np.sin(heading_angle), 0],
                         [np.sin(heading_angle),  np.cos(heading_angle), 0],
                         [                    0,                      0, 1]])
        t_wb = np.array([[Xs, Ys, Zs]])
        
        bbox_xyz = np.linalg.inv(R_wb) @ (world_xyz - t_wb).T
        bbox_xyz = bbox_xyz.T

        bbox_center_xyz = np.array([[Ls, Ws, Hs]]) / 2
        
        mask = (bbox_xyz[:,0]>=0) & (bbox_xyz[:,0]<=Ls) & \
               (bbox_xyz[:,1]>=0) & (bbox_xyz[:,1]<=Ws) & \
               (bbox_xyz[:,2]>=0) & (bbox_xyz[:,2]<=Hs)
        
        rescale_ratio = target_box[3:6]/source_box[3:6]
        bbox_rescale_xyz = bbox_center_xyz + (bbox_xyz[mask] - bbox_center_xyz) * rescale_ratio

        # bounding box coordinate system to world coordinate system
        world_new_xyz = R_wb @ bbox_rescale_xyz.T + t_wb.T
        world_new_xyz = world_new_xyz.T
        
#         print(source_box[0:3]-source_box[3:6]/2, source_box[0:3]+source_box[3:6]/2)
#         print(target_box[0:3]-target_box[3:6]/2, target_box[0:3]+target_box[3:6]/2)
        
#         print(np.min(world_xyz[mask], axis=0), np.max(world_xyz[mask], axis=0))
#         print(np.min(world_new_xyz, axis=0), np.max(world_new_xyz, axis=0))

        lidar['lidars']['points_xyz'][mask] = world_new_xyz
        
    # Save DA annotations
    with open(os.path.join(w2n_anno_dir, anno_file), 'wb') as anno_pk:
        pickle.dump(anno, anno_pk)

    # Save DA lidar
    with open(os.path.join(w2n_lidar_dir, anno_file), 'wb') as lidar_pk:
        pickle.dump(lidar, lidar_pk)

    break

  0%|                                                                                                                                                                                   | 0/158081 [00:00<?, ?it/s]


In [11]:
# Sanity check for an annotation
anno_file = anno_files[0]
print(anno_file)

with open(os.path.join(waymo_anno_dir, anno_file), 'rb') as anno_pk:
    anno_waymo = pickle.load(anno_pk)
with open(os.path.join(w2n_anno_dir, anno_file), 'rb') as anno_pk:
    anno_nuscenes = pickle.load(anno_pk)

for anno_waymo_obj,  anno_nuscenes_obj in zip(anno_waymo['objects'], anno_nuscenes['objects']):
    assert anno_waymo_obj['label'] == anno_nuscenes_obj['label'], "Label should be the same."
    label = anno_waymo_obj['label']

    if label not in waymo_labels:
        continue
    
    anno_waymo_obj['box'][3:6] = anno_waymo_obj['box'][3:6] + w2n_deltas[label]
    assert (anno_waymo_obj['box'] == anno_nuscenes_obj['box']).all(), "Coordinates are wrong."

seq_2_frame_0.pkl
